In [179]:
import json
import os
import requests
import time
import yaml


In [67]:
with open('/Users/nxo/Workspace/GitHub/arcsaef/config/reporting.yaml', 'r') as file:
        cf = yaml.safe_load(file)

In [174]:
# Highlight all journal articles of interest
# Run get_doi_ipop.js
# Save results as doi_org.txt
with open('/Users/nxo/Workspace/GitHub/arcsaef/data/all/doi_org.txt', mode= "r", encoding="utf-8") as f:
    doi_org_raw = f.readlines()

In [175]:
# Use doi a the key
# Use the values in ipop
doi_org = {}
for elem in doi_org_raw:
    if not elem.startswith("#"):
        if len(elem.split(":") ) == 2:
            doi_org[elem.split(":")[0].strip()] = elem.split(":")[1].strip()

In [176]:
nrpt_orgs = json.loads(cf['non-report']['organisations'])
rpt_orgs  = json.loads(cf['report']['organisations'])
nrpt_orgs.update(rpt_orgs)

for o in doi_org:
    for x in nrpt_orgs.keys():
        if x.upper() == doi_org.get(o):
            doi_org[o] = nrpt_orgs.get(x)

# handle output with >1 ipop
for o in doi_org:
    if doi_org.get(o).find(";") > -1:
        org_split = []
        for x in doi_org.get(o).split(";"):
            for p in nrpt_orgs.keys():
                if p.upper() == x.strip():
                    org_split.append(nrpt_orgs.get(x.strip()))
        doi_org[o] = org_split


In [177]:
url = "https://api.openalex.org/works/https://doi.org/"

for doi in doi_org:
    # there is no SAEF id that can be used to retrieve research output attributed to SAEF
    # therefore it is safer to use DOI's of known (recorded in Zotero) SAEF output 
    response = requests.get(f"{url}{doi}") 

    if response.status_code == 200:
        page  = json.loads(response.content) # dictionary 
        fname = f"{'/Users/nxo/Workspace/GitHub/arcsaef/data/jnls_202408/'}{page.get('id').split('/')[3]}.json"

        with open(fname, mode= "w", encoding="utf-8") as f:
            json.dump(page, f)

    time.sleep(1) # sleep for 1 second satisfies max 10 call per sec limit


In [2]:

with open('/Users/nxo/Workspace/GitHub/arcsaef/data/jnls_202408/W4392715378.json', mode= "r", encoding="utf-8") as f:
    work = json.load(f)

In [51]:
for author in work.get('authorships'):
    print(author.get('institutions')[0]['display_name'])

Queensland University of Technology
Queensland University of Technology
Queensland University of Technology
Queensland University of Technology
CSIRO Land and Water
Queensland University of Technology
Deakin University
Queensland University of Technology


In [189]:
files = []
for file in os.listdir('/Users/nxo/Workspace/GitHub/arcsaef/data/jnls_202408/'):
    files.append(file)

for f in files:
    with open(f"{'/Users/nxo/Workspace/GitHub/arcsaef/data/jnls_202408/'}{f}", mode= "r", encoding="utf-8") as f:
        work = json.load(f)

    print(doi_org.get(work.get('doi')[16: ]))
    # print(work.get('doi')[16: ] )

South Australia Museum
Monash University
Monash University
['James Cook University', 'Western Australia Museum']
University of Wollongong
Monash University
None
['University of Wollongong', 'Monash University']
Monash University
University of Wollongong
Monash University
Monash University
University of Wollongong
South Australia Museum
['Monash University', 'Berkeley Geochronology Center']
Monash University
None
Monash University
Monash University
University of Wollongong
None
Monash University
Monash University
['Monash University', 'University of Pretoria']
['University of Wollongong', 'University of the Balearic Islands']
University of Wollongong
Monash University
Monash University
Monash University
['La Trobe University', 'Monash University']
Monash University
Monash University
University of Wollongong
['Australian Nuclear Science & Technology Organisation', 'University of Wollongong']
None
Monash University
University of Wollongong
None
Monash University
Monash University
Monash U

In [168]:
doi_org

doi_org.get(work.get('doi')[16:-1])

{'10.1126/science.aay9165': 'University of Wollongong',
 '10.1007/s43630-020-00001-x': 'University of Wollongong',
 '10.1002/ece3.8376': ['James Cook University', 'Western Australia Museum'],
 '10.1073/pnas.2017384118': 'Monash University',
 '10.5194/tc-15-5447-2021': ['Monash University',
  'Berkeley Geochronology Center'],
 '10.1073/pnas.2025322118': 'Monash University',
 '10.1093/conphys/coab009': ['University of Wollongong', 'Monash University'],
 '10.1093/conphys/coab038': 'Monash University',
 '10.1111/gcb.15841': 'OUW',
 '10.1111/1365-2435.13829': ['La Trobe University', 'Monash University'],
 '10.1016/j.oneear.2022.05.009': 'La Trobe University',
 '10.1016/j.ympev.2022.107429': 'South Australia Museum',
 '10.1017/S0954102022000402': 'Queensland University of Technology',
 '10.1002/fee.2560': ['South Australia Museum',
  'Australian Antarctic Division'],
 '10.1038/s43017-022-00348-y': 'Monash University',
 '10.1016/j.agwat.2022.107695': 'University of Wollongong',
 '10.1111/gcb.

In [64]:
for x in doi_org:
    d = f"{'https://doi.org/'}{x}"
    if d == work['ids']['doi']:
        print( doi_org.get(x) )
        for author in work.get('authorships'):
            print(author.get('institutions')[0]['display_name'])

QUT
Queensland University of Technology
Queensland University of Technology
Queensland University of Technology
Queensland University of Technology
CSIRO Land and Water
Queensland University of Technology
Deakin University
Queensland University of Technology
